# Special topics: Web scraping

Web scraping is the process of capturing information from websites. Sometimes this is a simple as copying-and-pasting or downloading a file from the internet, which we have done many times. When it comes to coding, though, we are thinking about searching through websites for series of data sets we want to be able to capture in some sort of process.

[Here](https://blog.hartleybrody.com/web-scraping/) is a nice resource for this that we will use for parts of this topic.

In [2]:
from BeautifulSoup import BeautifulSoup
import requests

ImportError: No module named 'BeautifulSoup'

## 1. Data

### The data for today

Here is the example we'll be using for this topic: satellite data. A seminar speaker in Oceanography in fall 2015, Dr. Chuanmin Hu, shared with this department about some of his algorithms for satellite data processing. Satellite data gives an incredible spatial scale of information – literally the whole earth! – but requires quite a bit of clever processing to get out the data that researchers actually want to use. This becomes increasingly true the more as we collectively build up more complex algorithms to root out new perspectives, and to try to remove visual obstacles like clouds. Dr. Hu, in particular, has found a way to remove sun glint from some satellite data, which can be very useful if you care about a latitude where this tends to be a problem.

Dr. Hu's data is [hosted online](http://optics.marine.usf.edu/); we'll be using it today. Our goal is to select a data type and to then automate the process of downloading a year's worth of image files of that data.

### Fetching data in general

First, some general notes from our resource listed above:

> So the first thing you’re going to need to do is fetch the data. You’ll need to start by finding your “endpoints” — the URL or URLs that return the data you need.

> If you know you need your information organized in a certain way — or only need a specific subset of it — you can browse through the site using their navigation. Pay attention to the URLs and how they change as you click between sections and drill down into sub-sections.

> The other option for getting started is to go straight to the site’s search functionality. Try typing in a few different terms and again, pay attention to the URL and how it changes depending on what you search for. You’ll probably see a GET parameter like q= that always changes based on you search term.

> Try removing other unnecessary GET parameters from the URL, until you’re left with only the ones you need to load your data. Make sure that there’s always a beginning ? to start the query string and a & between each key/value pair.

### Fetching our data in particular

Let's click around on the website. First we should navigate to the Satellite Data Products. We'll try North America > Mississippi River. Notice the web address now:

`http://optics.marine.usf.edu/cgi-bin/optics_data?roi=MRIVER&current=1`

but also note that it doesn't change when we click on other time tabs or dates.

---
### *Exercise*

> Check out April 5, 2016 — there are a lot of neat, unobstructed datasets over several times for different satellite passes for this day. Explore the data. What happens when you click on an image?

---

### Data locations

So we've seen now that any particular satellite data image has its own unique address, for example:

`http://optics.marine.usf.edu/subscription/modis/MRIVER/2016/daily/096/A20160961915.QKM.MRIVER.PASS.L3D_RRC.RGB.png`

and another to compare with:

`http://optics.marine.usf.edu/subscription/modis/MRIVER/2016/daily/096/T20160961605.QKM.MRIVER.PASS.L3D.ERGB.png`

In order to automate downloading this data, we need to search out the unique addresses for each of the times it is available, loop over those addresses, and save the data — pretty simple! The difficult part is deconstructing the webpage in order to automate this process.

Let's take apart this image web address. We see that the first part, `http://optics.marine.usf.edu/subscription/`, is consistent between the two and looks like just a base address. These two images are from different satellites (MODIS-A and MODIS-T, respectively), but they both still have `modis` next in the address. We can guess that `MRIVER` is for Mississippi river, so it is selecting out this particular geographic region. Then we see the year, `2016`, and start getting into what is probably details about this particular file.

---
### *Exercise*

> To navigate through and understand the file system, go to `http://optics.marine.usf.edu/subscription/modis/MRIVER` and click around the links. What does each level of links refer to? How do you navigate to a single image file? What are all the different image files? Which one do you actually want, from the large list of image files you find?

> Out of this exercise, you should come away with a sample link of a particular kind of data you want to automate the selection of, and have an understanding of all the pieces of the web address, such that you know which parts of the address are consistent between different image addresses and what is distinct and would need to be looped over to capture all of these files.

---

## 2. The webpage

Now that we better understand the makeup of the web addresses we'll be using, we need to look at how to take apart the web site in order to dynamically access all of these files. That means, using the bit of knowledge we just gained, we want to be able to mine the necessary data file locations from the website itself with as few things hard-coded in as possible. (The more things we hard-code in, the more likely it is that a minor change on the website will break our code.)

We'll be using [requests](http://docs.python-requests.org/en/latest/index.html) and [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/) for this analysis. (For people who have done some of this before, note that Requests is now used preferentially over urllib and urllib2. More [here](http://stackoverflow.com/questions/2018026/what-are-the-differences-between-the-urllib-urllib2-and-requests-module).)

### Accessing the website

We use Requests to access the website:

In [ ]:
url = 'http://optics.marine.usf.edu/subscription/modis/' + args.area.upper() + '/' + str(args.year) + '/daily/'
soup = BeautifulSoup(requests.get(url).text)
